In [1]:
!where python

c:\Users\HK-Laptop-V639\Documents\GitHub\696\env696\Scripts\python.exe
C:\Users\HK-Laptop-V639\AppData\Local\Programs\Python\Python312\python.exe
C:\Users\HK-Laptop-V639\AppData\Local\Microsoft\WindowsApps\python.exe


In [2]:
# run this code to make sure you install all the required libraries
# be sure you are in virtual environment before install, otherwise it will overwrite your local environment

# !pip install -r requirements.txt

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
pd.set_option("display.max_columns", None)
RANDOM_SEED = 123

In [5]:
df_anime = pd.read_csv("assets/anime.csv")
df_manga = pd.read_csv("assets/manga.csv")

In [6]:
df_anime.shape, df_manga.shape

((24985, 39), (64833, 30))

## Data Prepocessing

Some short synopsis contain no information about the story of manga/title. This will introduce noise to our model. Therefore, we decide to remove those rows with extremely short synopsis.
Example:
- Second season of Mao Zhi Ming.
- The second season of Shen Lan Qi Yu Wushuang Zhu.
- Recap episode of Hakyuu Houshin Engi.
- Fifth Season of Bungou Stray Dogs
- 1-3. Ba_ku\n4-5. Mephisto
- An absurd film by Kuri Youji.
- Included one-shot:\nBougainvillea
- A collection of oneshots by Nishida Higashi.
- A movie adaptation of the TV series.
- Short film by Kurosaka Keita.
- Special episodes added to DVDs and Blu-rays.
- Movie based on the 1996 TV anime with an original plot.
- Third season of Yuan Long

In [7]:
def data_cleaning(input_anime, input_manga):
    df_anime = input_anime.copy()
    df_manga = input_manga.copy()

    # remove unnecessary columns
    df_anime = df_anime.drop(columns=['anime_id', 'total_duration', 'start_year', 'start_season', 'rating', 'main_picture', 'url', 'trailer_url', 'background', 'created_at', 'updated_at', 'episode_duration', 'broadcast_day', 'broadcast_time', 'licensors', 'title_synonyms', 'real_start_date', 'real_end_date'])
    df_manga = df_manga.drop(columns=['manga_id', 'main_picture', 'url', 'background', 'created_at_before', 'updated_at', 'title_synonyms', 'volumes', 'real_start_date', 'real_end_date'])

    # remove rows that are null in 'synopsis' and 'title', which are crucial for our project
    df_anime.dropna(subset=['title', 'synopsis', 'title_english', 'title_japanese'], inplace=True)
    df_manga.dropna(subset=['title', 'synopsis', 'title_english', 'title_japanese'], inplace=True)

    # remove '(Sources:...)' from synopsis
    df_anime['synopsis'] = df_anime['synopsis'].apply(lambda x: re.sub(r'\(Source:.*\)', '', x))
    df_manga['synopsis'] = df_manga['synopsis'].apply(lambda x: re.sub(r'\(Source:.*\)', '', x))

    # remove '[Written by ...]' from synopsis
    df_anime['synopsis'] = df_anime['synopsis'].apply(lambda x: re.sub(r'\[Written by.*\]', '', x))
    df_manga['synopsis'] = df_manga['synopsis'].apply(lambda x: re.sub(r'\[Written by.*\]', '', x))

    # remove rows that have extreme short synopsis
    df_anime = df_anime[df_anime['synopsis'].apply(lambda x: len(x) > 50)]
    df_manga = df_manga[df_manga['synopsis'].apply(lambda x: len(x) > 50)]

    print('cleaned anime shape: ', df_anime.shape)
    print('cleaned manga shape: ', df_manga.shape)

    return df_anime, df_manga

df_anime_cleaned, df_manga_cleaned = data_cleaning(df_anime, df_manga)

cleaned anime shape:  (8862, 21)
cleaned manga shape:  (15447, 20)


In [8]:
print("anime extra columns: \n", [col for col in df_anime_cleaned.columns if not col in df_manga_cleaned.columns])

anime extra columns: 
 ['episodes', 'source', 'studios', 'producers']


In [9]:
print("manga extra columns: \n", [col for col in df_manga_cleaned.columns if not col in df_anime_cleaned.columns])

manga extra columns: 
 ['chapters', 'authors', 'serializations']


In [10]:
print("common columns: \n", [col for col in df_anime_cleaned.columns if col in df_manga_cleaned.columns])

common columns: 
 ['title', 'type', 'score', 'scored_by', 'status', 'start_date', 'end_date', 'members', 'favorites', 'sfw', 'approved', 'genres', 'themes', 'demographics', 'synopsis', 'title_english', 'title_japanese']


In [11]:
import ast

def extract_author_name(author_list):
    try:
        author_list = ast.literal_eval(author_list)
        
        # Extract first and last names of authors, ignoring the others
        author_names = [f"{author['first_name']} {author['last_name']}".strip() for author in author_list if author['first_name'] or author['last_name']]
        
        # Join names for multiple authors
        return ', '.join(author_names) if author_names else "Missing"
    
    except (ValueError, SyntaxError, KeyError, TypeError):
        # Handle Missing data
        return "Missing"

# Apply the function to the 'authors' column
def authors_extraction(input_manga):
    df_manga = input_manga.copy()
    df_manga['authors'] = df_manga['authors'].apply(extract_author_name)

    return df_manga

df_manga_extracted = authors_extraction(df_manga_cleaned)

In [12]:
df_manga_cleaned['authors'].head(1).values

array(["[{'id': 1868, 'first_name': 'Kentarou', 'last_name': 'Miura', 'role': 'Story & Art'}, {'id': 49592, 'first_name': '', 'last_name': 'Studio Gaga', 'role': 'Art'}]"],
      dtype=object)

In [13]:
df_manga_extracted['authors'].head(1).values

array(['Kentarou Miura, Studio Gaga'], dtype=object)

**Extra columns alignment**:
| anime columns | manga columns | strategy |
| --- | --- | --- |
| episodes | chapters | episodes/chapters |
| source | NULL | impute const 'Missing' |
| studios | authors | creators |
| producers | serializations | production_source |

In [14]:
def columns_alignment(input_anime, input_manga):
    df_anime = input_anime.copy()
    df_manga = input_manga.copy()

    # treat 'episodes' and 'chapters' the same, create null value for 'volume' in anime
    df_anime.rename(columns={'episodes': 'episodes/chapters'}, inplace=True)
    df_manga.rename(columns={'chapters': 'episodes/chapters'}, inplace=True)

    # Combine studios and authors together to get creators columns
    df_anime.rename(columns={'studios': 'creators'}, inplace=True)
    df_manga.rename(columns={'authors': 'creators'}, inplace=True)

    # Also for producers and serialization
    df_anime.rename(columns={'producers': 'production_source'}, inplace=True)
    df_manga.rename(columns={'serializations': 'production_source'}, inplace=True)

    # To distinguish where the data from
    df_anime['is_anime'] = 1
    df_manga['is_anime'] = 0 
    
    return df_anime, df_manga

df_anime_aligned, df_manga_aligned = columns_alignment(df_anime_cleaned, df_manga_extracted)

In [15]:
df_full = pd.concat([df_anime_aligned, df_manga_aligned], ignore_index=True)

df_full.head()

,title,type,score,scored_by,status,episodes/chapters,start_date,end_date,source,members,favorites,sfw,approved,genres,themes,demographics,creators,production_source,synopsis,title_english,title_japanese,is_anime
0,Fullmetal Alchemist: Brotherhood,tv,9.10,2037075,finished_airing,64.0,2009-04-05,2010-07-04,manga,3206028,219036,True,True,"['Action', 'Adventure', 'Drama', 'Fantasy']",['Military'],['Shounen'],['Bones'],"['Aniplex', 'Square Enix', 'Mainichi Broadcast...",After a horrific alchemy experiment goes wrong...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,1
1,Hunter x Hunter (2011),tv,9.04,1671587,finished_airing,148.0,2011-10-02,2014-09-24,manga,2688079,202109,True,True,"['Action', 'Adventure', 'Fantasy']",[],['Shounen'],['Madhouse'],"['VAP', 'Nippon Television Network', 'Shueisha']",Hunters devote themselves to accomplishing haz...,Hunter x Hunter,HUNTER×HUNTER（ハンター×ハンター）,1
2,Shingeki no Kyojin Season 3 Part 2,tv,9.05,1491491,finished_airing,10.0,2019-04-29,2019-07-01,manga,2133927,55644,True,True,"['Action', 'Drama']","['Gore', 'Military', 'Survival']",['Shounen'],['Wit Studio'],"['Production I.G', 'Dentsu', 'Mainichi Broadca...",Seeking to restore humanity's diminishing hope...,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,1
3,Steins;Gate,tv,9.07,1348232,finished_airing,24.0,2011-04-06,2011-09-14,visual_novel,2463954,184312,True,True,"['Drama', 'Sci-Fi', 'Suspense']","['Psychological', 'Time Travel']",[],['White Fox'],"['Frontier Works', 'Media Factory', 'Kadokawa ...",Eccentric scientist Rintarou Okabe has a never...,Steins;Gate,STEINS;GATE,1
4,Koe no Katachi,movie,8.94,1540277,finished_airing,1.0,2016-09-17,2016-09-17,manga,2218467,84124,True,True,"['Award Winning', 'Drama']",['Romantic Subtext'],['Shounen'],['Kyoto Animation'],"['Shochiku', 'Pony Canyon', 'Kodansha', 'ABC A...","As a wild youth, elementary school student Sho...",A Silent Voice,聲の形,1


### Date Transformation

In [16]:
# Function to extract year and month
def extract_year_and_month(date):
    try:
        # Convert the date string to a datetime object
        datetime_date = pd.to_datetime(date, errors='raise')
        # Extract year and month
        return datetime_date.year, datetime_date.month
    except:
        # return NaN if fails
        return np.nan, np.nan
    
# Apply the function to the 'start_date' column
df_full[['start_year', 'start_month']] = df_full['start_date'].apply(lambda x: pd.Series(extract_year_and_month(x)))

# Apply the function to the 'end_date' column
df_full[['end_year', 'end_month']] = df_full['end_date'].apply(lambda x: pd.Series(extract_year_and_month(x)))

#remove useless columns
df_full = df_full.drop(columns=['start_date', 'end_date'])

In [17]:
# Function to transform Start or End month into season refer to Events of Anime
def month_to_season(month):
    # If the data is in range, return corresponding Season of events of Anime
    if month in [1, 2, 3]:
        return 'Winter'
    elif month in [4, 5, 6]:
        return 'Spring'
    elif month in [7, 8, 9]:
        return 'Summer'
    elif month in [10, 11, 12]:
        return 'Autumn'
    else:
        return np.nan  # Handle unexpected cases, though this shouldn't occur with valid months

# Apply the function to transform the month value to season categories
df_full['start_season'] = df_full['start_month'].apply(month_to_season)
df_full['end_season'] = df_full['end_month'].apply(month_to_season)

# convert year to int
df_full['start_year'] = df_full['start_year'].astype('Int64')
df_full['end_year'] = df_full['end_year'].astype('Int64')

# remove useless columns
df_full = df_full.drop(columns=['start_month', 'end_month'])

# Check for Year and Season feature
df_full[['start_year', 'end_year', 'start_season', 'end_season']].head()

,start_year,end_year,start_season,end_season
0,2009,2010,Spring,Summer
1,2011,2014,Autumn,Summer
2,2019,2019,Spring,Summer
3,2011,2011,Spring,Summer
4,2016,2016,Summer,Summer


### Train Test data split

In [18]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_full, test_size=0.2, random_state=RANDOM_SEED)
print('Number of rows in train set: ', len(train))

Number of rows in train set:  19447


In [19]:
# check for missing data 
print("Number of missing data in Training Set:\n")
print(train.isnull().sum())

Number of missing data in Training Set:

title                    0
type                     6
score                 5962
scored_by                0
status                   0
episodes/chapters     3898
source               12984
members                  0
favorites                0
sfw                      0
approved                 0
genres                   0
themes                   0
demographics             0
creators                 0
production_source        0
synopsis                 0
title_english            0
title_japanese           0
is_anime                 0
start_year             324
end_year              3490
start_season           324
end_season            3490
dtype: int64


In [20]:
# check for missing data 
print("Number of missing data in Testing Set:\n")
print(test.isnull().sum())

Number of missing data in Testing Set:

title                   0
type                    2
score                1496
scored_by               0
status                  0
episodes/chapters     959
source               3237
members                 0
favorites               0
sfw                     0
approved                0
genres                  0
themes                  0
demographics            0
creators                0
production_source       0
synopsis                0
title_english           0
title_japanese          0
is_anime                0
start_year             79
end_year              863
start_season           79
end_season            863
dtype: int64


### Handle Missing data

In [21]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# Non-numerical columns
Non_Numerical = ['type', 'source', 'start_season', 'end_season']

# Create an imputer for non-numerical data filling with 'Missing'
categorical_imputer = SimpleImputer(strategy='constant', fill_value='Missing')

# Impute non-numerical columns
train[Non_Numerical] = categorical_imputer.fit_transform(train[Non_Numerical])

# Numerical columns
Numerical = ['score', 'scored_by', 'episodes/chapters', 'members', 'favorites', 'start_year' , 'end_year']

# Create an imputer for numerical data using KNNImputer
knn_imputer = KNNImputer(n_neighbors=3)

# Impute numerical columns
train[Numerical] = knn_imputer.fit_transform(train[Numerical])

# Check for result
print("Number of missing data in Training Set:\n")
print(train.isnull().sum())

Number of missing data in Training Set:

title                0
type                 0
score                0
scored_by            0
status               0
episodes/chapters    0
source               0
members              0
favorites            0
sfw                  0
approved             0
genres               0
themes               0
demographics         0
creators             0
production_source    0
synopsis             0
title_english        0
title_japanese       0
is_anime             0
start_year           0
end_year             0
start_season         0
end_season           0
dtype: int64


### Data Normalization

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Selecting columns to normalize
columns_to_normalize = ["scored_by", "episodes/chapters", "members", "favorites"]

earliest_start_year = train['start_year'].min()

# function for Normalizing the selected columns
def Normalizing(data):
    data_normalized = data.copy()
    data_normalized[columns_to_normalize] = scaler.fit_transform(train[columns_to_normalize])

    # Calculate the duration by subtracting the earliest start year from all years
    data_normalized['elapsed_start_time '] = data_normalized['start_year'] - earliest_start_year
    data_normalized['elapsed_end_time '] = data_normalized['end_year'] - earliest_start_year

    data_normalized = data_normalized.drop(columns=['start_year', 'end_year'])

    return data_normalized

train_normalized = Normalizing(train)
train_normalized.head()

,title,type,score,scored_by,status,episodes/chapters,source,members,favorites,sfw,approved,genres,themes,demographics,creators,production_source,synopsis,title_english,title_japanese,is_anime,start_season,end_season,elapsed_start_time,elapsed_end_time
7048,Toyama Kankou Anime Project,special,5.250000,-0.198164,finished_airing,-0.394637,original,-0.228160,-0.114920,True,True,['Slice of Life'],['Historical'],[],['P.A. Works'],['The Berich'],The Toyama Kankou Anime Project is a visualiza...,Toyama Tourism Anime Project,富山観光アニメプロジェクト,1,Spring,Spring,92.0,92.000000
2307,Slayers: The Motion Picture,movie,7.270000,-0.065672,finished_airing,-0.496178,light_novel,-0.086360,-0.109830,True,True,"['Adventure', 'Comedy', 'Fantasy']",[],[],['J.C.Staff'],"['Kadokawa Shoten', 'Marubeni']",In this prequel movie to the Slayers televison...,Slayers: The Motion Picture,劇場版スレイヤーズ,1,Summer,Summer,78.0,78.000000
9709,Arte,manga,7.910000,-0.170972,currently_publishing,0.810314,Missing,-0.159651,-0.069930,True,True,[],"['Historical', 'Visual Arts']",['Seinen'],Kei Ookubo,['Comic Zenon'],"It is early 16th-century Italy, and the city o...",Arte,アルテ,0,Autumn,Missing,96.0,101.000000
23892,Maou to Yuusha no Tatakai no Ura de: Game Seka...,light_novel,6.556667,-0.202049,currently_publishing,-0.083245,Missing,-0.236136,-0.115327,True,True,"['Action', 'Fantasy']","['Isekai', 'Reincarnation', 'Video Game']",[],"Sanshouuo, Yuuki Suzuki",[],A modern man is reincarnated into the world of...,Reincarnated into a Game as the Hero's Friend:...,魔王と勇者の戦いの裏で ～ゲーム世界に転生したけど友人の勇者が魔王討伐に旅立ったあとの国内お...,0,Winter,Missing,105.0,104.666667
15217,The Devil's Temptation,manhwa,6.980000,-0.199772,finished,1.737720,Missing,-0.232042,-0.114920,False,True,"['Boys Love', 'Erotica', 'Supernatural']",[],[],Youn,['Lezhin Comics Webtoon'],Hyun was supposed to spend Christmas snuggling...,The Devil's Temptation,악마의 유혹,0,Spring,Winter,101.0,105.000000


### Text processing

Tokenization and lemmatization by keeping `{'NOUN', 'VERB', 'ADJ', 'PROPN', 'ADV'}` and replace named entity with place holder.

In [23]:
# tokenization and lemmatization
import spacy

POS_TO_KEEP = {'NOUN', 'VERB', 'ADJ', 'PROPN', 'ADV'}

nlp = spacy.load('en_core_web_sm')

stop_words = nlp.Defaults.stop_words

def tokenization(doc):
    """
    Filter out number.
    Replace person, organization, and location entities with '<ent_type_>'
    Return lemma if its POS is in `POS_TO_KEEP` and not in stop_words.
    """
    tokens = []

    for token in doc:
        if token.is_digit:
            # filter out numeric tokens
            continue

        if token.ent_type_ in ['PERSON', 'ORG', 'GPE']:
            # replace person, organization, and location entities
            tokens.append(f'<{token.ent_type_}>')
            
        elif token.pos_ in POS_TO_KEEP and not token.lemma_ in stop_words:
            # return lemma if its POS is in `POS_TO_KEEP` and not in stop_words
            tokens.append(token.lemma_)
    
    return tokens

# use nlp.pipe for batch processing
train['title_en_token'] = [
    tokenization(doc) for doc in nlp.pipe(train['title_english'], batch_size=100, n_process=-1)
]
train['synopsis_token'] = [
    tokenization(doc) for doc in nlp.pipe(train['synopsis'], batch_size=100, n_process=-1)
]

# filter out empty token
train = train[train['title_en_token'].apply(lambda x: len(x) > 0) & train['synopsis_token'].apply(lambda x: len(x) > 0)] # both columns cannot be empty

print('Number of rows after filtering empty token: ', len(train))

Number of rows after filtering empty token:  19301


In [24]:
# review tokenization
with pd.option_context('display.max_colwidth', None):
    display(train[['title_english', 'title_en_token', 'synopsis', 'synopsis_token']].sample(1))

,title_english,title_en_token,synopsis,synopsis_token
9075,Teasing Master Takagi-san,"[Teasing, Master, Takagi, san]","With a smirk, the mischievous middle school boy Nishikata glances over at Takagi, the girl seated next to him in class. He has just devised a master plan to finally get back at her for all the merciless teasing inflicted upon him. As he is about to set his plan into motion, Takagi intervenes with a single comment that halts Nishikata right in his tracks. She had turned the tables on him yet again.\n \n""If you blush, you lose."" That has been the unwritten rule set between the two ever since they encountered one another in middle school. Day after day, loss after loss, Nishikata strives to see Takagi red with embarrassment, but his futile attempts are only met by further ridicule. Beyond this vicious cycle of trying to outwit one another, will their relationship ever evolve?\n\n","[smirk, mischievous, middle, school, boy, <ORG>, glance, <ORG>, girl, seat, class, devise, master, plan, finally, merciless, teasing, inflict, set, plan, motion, <ORG>, intervene, single, comment, halt, <ORG>, right, track, turn, table, blush, lose, unwritten, rule, set, encounter, middle, school, day, day, loss, loss, <ORG>, strive, <ORG>, red, embarrassment, futile, attempt, meet, ridicule, vicious, cycle, try, outwit, relationship, evolve]"


Since titles and synopses play different roles (titles are short and often genre-indicative, while synopses provide detailed content descriptions), we use two separate vectorizers.

Titles are shorter and often contain rare, context-rich words that are crucial for capturing unique meaning, while synopses are longer and contain more common words, making them less reliant on capturing rare vocabulary. So, we use a higher `max_features` or `vector_size` for title to ensures it capture these niche terms and their relationships, while use lower for synopsis to focus on the more frequently relevant words.

In [25]:
# tfidf

from sklearn.feature_extraction.text import TfidfVectorizer

# initialize title tfidf
tfidf_title = TfidfVectorizer(
    ngram_range=(1,1),  # uni-gram
    min_df=1,           # don't filter rare words as they are important for title
    max_df=0.8,         # filter out very common words
)

# initialize synopsis tfidf
tfidf_synopsis = TfidfVectorizer(
    ngram_range=(1,2),  # uni-gram or bi-gram
    max_features=2000,  # focus on the more frequently relevant words
    min_df=2,           # filter out extremely rare words
    max_df=0.8,         # filter out very common words
)

# train tfidf
title_en_tfidf_matrix = tfidf_title.fit_transform(train['title_en_token'].apply(lambda x: " ".join(x)))
synopsis_tfidf_matrix = tfidf_synopsis.fit_transform(train['synopsis_token'].apply(lambda x: " ".join(x)))

# add result to train df
train['title_en_tfidf'] = [title_en_tfidf_matrix[i] for i in range(title_en_tfidf_matrix.shape[0])]
train['synopsis_tfidf'] = [synopsis_tfidf_matrix[i] for i in range(synopsis_tfidf_matrix.shape[0])]

For word embedding, we choose **Word2Vec - Skip-gram** because it tends to capture rare words more effectively (e.g. niche anime/manga-specific vocabulary), comparing to Word2Vec - CBOW and GloVe.

In [26]:
# word embedding
import multiprocessing
from gensim.models import Word2Vec

# dynamically determine the number of CPU cores
num_workers = multiprocessing.cpu_count()

# train title skipgram model
skipgram_model_title = Word2Vec(
    train['title_en_token'].tolist(),
    sg=1,               # skip-gram
    vector_size=300,    # title use higher dim
    window=2,           # title use smaller window size
    min_count=1,        # titles may contain rare but important words
    epochs=30,          # title are shorter, need more epochs to train
    workers=num_workers,
    seed=RANDOM_SEED
)

# train synopsis skipgram model
skipgram_model_synopsis = Word2Vec(
    train['synopsis_token'].tolist(),
    sg=1,               # skip-gram
    vector_size=150,    # synopsis use lower dim
    window=5,           # synopsis use larger window size
    min_count=2,        # filter out extremely rare words
    epochs=15,
    workers=num_workers,
    seed=RANDOM_SEED
)

# clean tokens that does not exist in the skipgram vocab (because of `min_count`)
model_vocab = set(skipgram_model_synopsis.wv.index_to_key)
train['synopsis_token'] = train['synopsis_token'].apply(lambda x: [token for token in x if token in model_vocab])
train = train[train['synopsis_token'].apply(lambda x: len(x) > 0)]  # filter out empty entry after clean tokens

# apply skipgram model
train['title_en_skipgram'] = train['title_en_token'].apply(lambda x: skipgram_model_title.wv[x])
train['synopsis_skipgram'] = train['synopsis_token'].apply(lambda x: skipgram_model_synopsis.wv[x])

In [27]:
train[['title_japanese', 'title_english', 'synopsis', 'title_en_tfidf', 'title_en_skipgram', 'synopsis_tfidf', 'synopsis_skipgram']].sample(5)

,title_japanese,title_english,synopsis,title_en_tfidf,title_en_skipgram,synopsis_tfidf,synopsis_skipgram
22683,アザトメイキング＋,Azato Making+,1. Shuujuu Emotion\n2. Shuujuu Dreaming\n3. Sh...,"(0, 5387)\t1.0","[[-0.5122074, -0.1496037, 0.0459687, -0.406966...","(0, 1149)\t0.25654260098412046\n (0, 1216)\...","[[0.07682086, -0.14349373, 0.04345498, 0.18370..."
10318,キミに恋していーですか。,May I be in love with you?,Chigira Ruiji is a man born with the face of a...,"(0, 4365)\t1.0","[[-0.009436031, -0.3950392, 0.030088374, -0.30...","(0, 1149)\t0.1875630314123266\n (0, 782)\t0...","[[0.07682086, -0.14349373, 0.04345498, 0.18370..."
22950,소년들은 무엇을 하고 있을까,10th Dimension Boys,"In another dimension, anything can happen in y...","(0, 925)\t0.41877466769782945\n (0, 1974)\t...","[[-0.086346425, -0.07495511, 0.10810716, -0.07...","(0, 707)\t0.19858748620401637\n (0, 936)\t0...","[[0.07777373, -0.31890902, -0.44942433, 0.4214..."
15893,ピュア・トランス,Pure Trance,"In this dreamy science fiction fantasy, Junko ...","(0, 5631)\t1.0","[[-0.06057793, -0.35592693, 0.06409358, -0.183...","(0, 1149)\t0.09993168318485023\n (0, 1216)\...","[[0.20115462, -0.011873402, -0.45442194, -0.03..."
8428,巨人の星対鉄腕アトム,Star of the Giants vs Astro Boy,Star of the Giants vs Astro Boy is an anime cr...,"(0, 5387)\t0.3197068523302739\n (0, 5631)\t...","[[-0.045020744, -0.073905714, 0.17955288, 0.01...","(0, 1149)\t0.4376688511097766\n (0, 1216)\t...","[[-0.064565, -0.39407685, -0.3785122, 0.230841..."


## Export necessary assets

In [28]:
import joblib

# train and test df
joblib.dump(train, 'assets/train.joblib')
joblib.dump(test, 'assets/test.joblib')

# tfidf_matrix
joblib.dump(title_en_tfidf_matrix, 'assets/title_en_tfidf_matrix.joblib')
joblib.dump(synopsis_tfidf_matrix, 'assets/synopsis_tfidf_matrix.joblib')

# tfidf vectorizer
joblib.dump(tfidf_title, 'assets/tfidf_title_vectorizer.joblib')
joblib.dump(tfidf_synopsis, 'assets/tfidf_synopsis_vectorizer.joblib')

# skipgram
joblib.dump(skipgram_model_title, 'assets/skipgram_model_title.joblib')
joblib.dump(skipgram_model_synopsis, 'assets/skipgram_model_synopsis.joblib')

['assets/skipgram_model_synopsis.joblib']

In [29]:
# store library version
# run every time before you commit
!pip freeze > requirements.txt